In [2]:
import os,sys
import glob
import numpy as np
from tqdm.notebook import tqdm_notebook
import umap
from scipy.io import loadmat
from os.path import exists
import h5py

phaseMain = "Test"

In [3]:
DLfeats_archive = np.load('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\DLfeats\\dlFeats.npz')

In [ ]:
critDLphase = np.char.find(DLfeats_archive['Dir'],phaseMain)>=0
DLFeats = DLfeats_archive['Feats'][critDLphase]
DLdir11 = DLfeats_archive['Dir'][critDLphase]

In [4]:
# DLdir
np.any(np.isinf(DLFeats))

False

In [5]:
DLdir1 = glob.glob(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\OrganSiteClassifying\\{phaseMain}\\**\\*.png')
DLdir = np.array(DLdir1)

In [6]:
np.array_equal(DLdir11, DLdir, equal_nan=False)

True

In [1]:
files=glob.glob(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\**\\**\\*_allFeats.mat')
handExist = np.zeros((len(DLdir),))
handExist = handExist==0
for i in tqdm_notebook(range(len(DLdir))):
    fn = DLdir[i]
    fn_split =  DLdir[i].split('\\')
    patchName =  DLdir[i].split('\\')[-1][:-4]
    existance = exists(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\{fn_split[-2]}\\{patchName}\\{fn_split[-1]}_allFeats.mat')
    if existance:
        arrays = {}
        f = h5py.File(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\{fn_split[-2]}\\{patchName}\\{fn_split[-1]}_allFeats.mat','r')
        for k, v in f.items():
            arrays[k] = np.array(v)
        f.close()
        if len(arrays['allFeats'][0])!=987:
            os.remove(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\{fn_split[-2]}\\{patchName}\\{fn_split[-1]}_allFeats.mat')
    handExist[i] = exists(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\{fn_split[-2]}\\{patchName}\\{fn_split[-1]}_allFeats.mat')

NameError: name 'glob' is not defined

In [8]:
# files=glob.glob(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\**\\**\\*_allFeats.mat')
len(files)
handExist

array([ True,  True,  True, ...,  True,  True,  True])

In [9]:
DLFeats = DLFeats[handExist,:]
DLdir = DLdir[handExist]
DLdir.shape

(18615,)

In [10]:
mapper = umap.UMAP(random_state=42,n_components=300 )
Dlembed = mapper.fit_transform(DLFeats)

In [11]:
Dlembed.shape

(18615, 300)

In [12]:
f = h5py.File('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\keptFeats.mat','r')
arrays = {}
for k, v in f.items():
    arrays[k] = np.array(v)
f.close()
critHandKept = arrays['crit'][0]==1

In [13]:
topHCfeatIdx = np.array([261,197,210, 145, 120, 84, 66, 36, 285,212])

In [14]:
pooledFeats = np.zeros((len(DLdir),10))
critNanInf = np.zeros((len(DLdir),), dtype=bool)
for i in tqdm_notebook(range(len(DLdir))):
    fn = DLdir[i]
    fn_split =  DLdir[i].split('\\')
    patchName =  DLdir[i].split('\\')[-1][:-4]
    arrays = {}
    f = h5py.File(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\{fn_split[-2]}\\{patchName}\\{fn_split[-1]}_allFeats.mat','r')
    for k, v in f.items():
        arrays[k] = np.array(v)
    f.close()
    if len(arrays['allFeats'][0])!=987:
        os.remove(f'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\HandcraftFeats\\{phaseMain}\\{fn_split[-2]}\\{patchName}\\{fn_split[-1]}_allFeats.mat')
        continue
    handFeatsAll = arrays['allFeats'][0][critHandKept].reshape(376,1)
    handFeats = handFeatsAll[topHCfeatIdx]
    critNanInf[i] = np.any(np.isinf(handFeats)) or np.any(np.isnan(handFeats))
#     dlEmbedFeats = Dlembed[i,:].reshape(1,10)
    pooledFeats[i,:] = handFeats.T

In [15]:
critInclude = np.logical_not(critNanInf) 
pooledFeats = pooledFeats[critInclude,:]
print(np.any(np.isnan(pooledFeats)) or np.any(np.isinf(pooledFeats)))


False


In [16]:
Dlembed = Dlembed[critInclude,:]
DLdir = DLdir[critInclude]
print(pooledFeats.shape)
print(Dlembed.shape)
print(DLdir.shape)

(14503, 10)
(14503, 300)
(14503,)


In [17]:
print(pooledFeats.shape)
print(Dlembed.shape)
print(DLdir.shape)

(14503, 10)
(14503, 300)
(14503,)


In [18]:
nameDir = [] 
for i in DLdir:
    nn = i.split('\\')
    nameDir.append(nn[-1])
    

In [19]:
len(nameDir)

14503

In [20]:
name_dictionary = dict(zip(nameDir, pooledFeats))
name_dictionary

{'20M_104449_45057_15.png': array([-1.23703894e-06, -2.22903528e-02,  4.04275110e-04,  9.73546198e-01,
         3.34114894e-01,  1.84748082e+01,  2.07183095e+01,  9.74101027e-01,
        -4.11830981e-01,  1.98003817e-01]),
 '20M_104449_45057_22.png': array([-9.37968617e-08,  8.54116665e-02,  2.36754793e-03,  9.84193448e-01,
         2.35996119e-01,  1.87086188e+01,  1.90811769e+01,  9.59242276e-01,
        -3.49158188e-01,  1.57508879e-01]),
 '20M_104449_45057_23.png': array([-3.40371731e-08, -1.09022147e+00, -2.52565360e-04,  9.78645609e-01,
         3.92374763e-01,  1.61271189e+01,  1.76587803e+01,  9.48697925e-01,
        -2.56810741e+00,  1.71270172e-01]),
 '20M_104449_45057_24.png': array([-2.05400030e-06, -3.77751928e-01,  1.58838950e-03,  9.78340391e-01,
         3.28635974e-01,  1.76013543e+01,  1.75206454e+01,  9.65503452e-01,
        -3.32442670e-01,  1.83017460e-01]),
 '20M_104449_45057_30.png': array([-2.16556827e-06, -8.72054972e-01, -1.14818208e-03,  9.78194005e-01,
     

In [21]:
if phaseMain == "Test":
    nameCase = '20M_104449_45057_15.png'
else:
    nameCase = '21M_14337_45057_13.png'

In [22]:
name_dictionary[nameCase].shape

(10,)

In [23]:
name_dictionary_DLfeats = dict(zip(nameDir, Dlembed))


In [24]:
name_dictionary_DLfeats[nameCase].shape

(300,)

In [25]:
#v3.classification
#28/11/2018

dataname="CUPBinlinFusion"
patch_size=256 #size of the tiles to extract and save in the database, must be >= to training size
stride_size=256 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
mirror_pad_size=128 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
test_set_size=.3 # what percentage of the dataset should be used as a held out validation/testing set
resize=1 #resize input images
class_names=["Colon","Esophagus", "Breast",  "Pancreas"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes
# class_names=["Pri","Met"]
#-----Note---
#One should likely make sure that  (nrow+mirror_pad_size) mod patch_size == 0, where nrow is the number of rows after resizing
#so that no pixels are lost (any remainer is ignored)

In [26]:
import torch
import tables

import PIL

import cv2
import matplotlib.pyplot as plt

from sklearn import model_selection
import sklearn.feature_extraction.image
import random


seed = random.randrange(sys.maxsize) #get a random seed so that we can reproducibly do the cross validation setup
random.seed(7654788126099255772) # set the seed
print(f"random seed (note down for reproducibility): {seed}")

random seed (note down for reproducibility): 5208618228519826049


In [27]:
img_dtype = tables.UInt8Atom()  # dtype in which the images will be saved, this indicates that images will be saved as unsigned int 8 bit, i.e., [0,255]
filenameAtom = tables.StringAtom(itemsize=255) #create an atom to store the filename of the image, just incase we need it later, 
hc_dtype = tables.Float32Atom()

In [28]:
# files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\PvM\\Test\\**\\*.png') # create a list of the files, in this case we're only interested in files which have masks so we can use supervised learning
# files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\Heterogenicity\\ReducePatchSize\\Patch256By256\\**\\*.png')
#files2=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Train\\Met\\*.png')
#files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Test\\Pri\\*.png') # create a list of the files, in this case we're only interested in files which have masks so we can use supervised learning
#files2=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Test\\Met\\*.png')
#files.extend(files2)
# files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\PvM\\WSI\\**\\*.png')
files=DLdir
random.shuffle(files)

In [29]:
files[0].split('\\')

['E:',
 'MvP',
 'FinalExperimentWithPancreas',
 'NewRNG',
 'Run3',
 'DeepLearningHeatmap',
 'OrganSiteClassifying',
 'Test',
 'Esophagus',
 '44M (2)_124929_51201_31.png']

In [30]:
print(len(files))

14503


In [31]:
#create training and validation stages and split the files appropriately between them
phases={}
if phaseMain == "Train":
    
    phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))
else:
    phases["Test"],phases["Test2"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# phases["WSI"],phases["WSI1"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))

In [32]:
# phases["val"].shape

In [33]:
np.append([0],name_dictionary[nameCase].shape)

array([ 0, 10])

In [34]:
block_shape=np.array((patch_size,patch_size,3))
np.append([0],block_shape)

array([  0, 256, 256,   3])

In [35]:
name_dictionary_DLfeats[nameCase].shape

(300,)

In [36]:
#--subset for rapid testing
#phases["train"]=phases["train"][0:100]
#phases["val"]=phases["val"][0:20]
from tqdm import tqdm_notebook

In [37]:
storage={} #holder for future pytables
hc_shape = name_dictionary[nameCase].shape
dl_shape = name_dictionary_DLfeats[nameCase].shape
block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed

for phase in phases.keys(): #now for each of the phases, we'll loop through the files
#     print(phase)
    
    totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    
    hdf5_file = tables.open_file(f".\\{dataname}_{phase}.pytable", mode='w') #open the respective pytable
    storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    
    storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
                                              shape=np.append([0],block_shape), 
                                              chunkshape=np.append([1],block_shape),
                                              filters=filters)
    storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
                                              shape=[0], 
                                              chunkshape=[1],
                                              filters=filters)
    
    storage["HCFeats"]= hdf5_file.create_earray(hdf5_file.root, "HCFeats", hc_dtype,  
                                              shape=np.append([0],hc_shape), 
                                              chunkshape=np.append([1],hc_shape),
                                              filters=filters)
    storage["DLfeats"]= hdf5_file.create_earray(hdf5_file.root, "DLfeats", hc_dtype,  
                                              shape=np.append([0],dl_shape), 
                                              chunkshape=np.append([1],dl_shape),
                                              filters=filters)
        
    for filei in tqdm_notebook(phases[phase]): #now for each of the files
        fname=files[filei]
        nnSplit = fname.split('\\')
        patchName = nnSplit[-1]
        hc=name_dictionary[patchName]
        dl = name_dictionary_DLfeats[patchName]
#         print(fname)
        classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
        totals[classid]+=1

        io=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
        interp_method=PIL.Image.BICUBIC


        io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
        #io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

        #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
        io_arr_out=sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)

        #resize it into a ntile x patch_size x patch_size x 3
        io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)
        hc=hc.reshape(1,len(name_dictionary[nameCase]))
        dl = dl.reshape(1,len(name_dictionary_DLfeats[nameCase]))



        storage["imgs"].append(io_arr_out)
        
        storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        storage["HCFeats"].append(hc)
        storage["DLfeats"].append(dl)
        storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        
    #lastely, we should store the number of pixels
    npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
    npixels[:]=totals
    hdf5_file.close()


#     for filei in tqdm_notebook(phases[phase]): #now for each of the files
#         fname=files[filei] 
#         nnSplit = fname.split('\\')
#         patchName = f'{nnSplit[-2]}.png'
#         classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
#         totals[classid]+=1

#         io=name_dictionary[patchName]
#         interp_method=PIL.Image.BICUBIC


# #         io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
#         #io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

#         #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
#         io_arr_out=sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size),stride_size)

#         #resize it into a ntile x patch_size x patch_size x 3
#         io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size)



#         storage["imgs"].append(io_arr_out)
#         storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
#         storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        
#     #lastely, we should store the number of pixels
#     npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
#     npixels[:]=totals
#     hdf5_file.close()

e:\mvp\venv\lib\site-packages\ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


e:\mvp\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function extract_patches is deprecated; The function feature_extraction.image.extract_patches has been deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# hc

In [ ]:
# hc.reshape(1,10)

In [ ]:
hdf5_file.close()
#useful reference
#http://machinelearninguru.com/deep_learning/data_preparation/hdf5/hdf5.html

In [ ]:
print(f'Test Size:{len(phases["val"])}')
print(f'Training Size:{len(phases["train"])}')

In [ ]:
valTable = tables.open_file(f'{dataname}_Test.pytable')
trainTable = tables.open_file(f'{dataname}_train.pytable')

In [ ]:
valTable

In [ ]:
trainTable

In [ ]:
table = valTable.root.HCFeats
asv = valTable.root.labels
acs = valTable.root.DLfeats

In [ ]:
table

In [ ]:
acs

In [ ]:

# dataname="CUP-test"

# patch_size=256 #size of the tiles to extract and save in the database, must be >= to training size
# stride_size=256 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
# mirror_pad_size=128 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
# test_set_size=.3 # what percentage of the dataset should be used as a held out validation/testing set
# resize=1 #resize input images
# # class_names=["Colon","Esophagus", "Breast",  "Pancreas"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes
# class_names=["Pri","Met"]
# storage={} #holder for future pytables

# block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

# filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed

# files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\PvM\\WSI\\**\\*.png')

# random.shuffle(files)
# phases={}
# # phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))
# # phases["Test"],phases["Test2"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# phases["WSI"],phases["WSI1"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# for phase in phases.keys(): #now for each of the phases, we'll loop through the files
# #     print(phase)
    
#     totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    
#     hdf5_file = tables.open_file(f".\\{dataname}_{phase}.pytable", mode='w') #open the respective pytable
#     storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    
#     storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
#                                               shape=np.append([0],block_shape), 
#                                               chunkshape=np.append([1],block_shape),
#                                               filters=filters)
#     storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
#                                               shape=[0], 
#                                               chunkshape=[1],
#                                               filters=filters)
#     storage["HCFeats"]= hdf5_file.create_earray(hdf5_file.root, "HCFeats", img_dtype,  
#                                               shape=[0], 
#                                               chunkshape=[10],
#                                               filters=filters)

    
#     for filei in tqdm_notebook(phases[phase]): #now for each of the files
#         fname=files[filei] 
        
# #         print(fname)
#         classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
#         totals[classid]+=1

#         io=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
#         interp_method=PIL.Image.BICUBIC


#         io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
#         #io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

#         #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
#         io_arr_out=sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)

#         #resize it into a ntile x patch_size x patch_size x 3
#         io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)



#         storage["imgs"].append(io_arr_out)
#         storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
#         storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        
        
#     #lastely, we should store the number of pixels
#     npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
#     npixels[:]=totals
#     hdf5_file.close()

In [ ]:
# io_arr_out.shape

In [ ]:
# print(f'Test Size:{len(phases["val"])}')
# print(f'Training Size:{len(phases["train"])}')

In [ ]:
# valTable = tables.open_file(f'{dataname}_val.pytable')
# trainTable = tables.open_file(f'{dataname}_train.pytable')

In [ ]:
# valTable

In [ ]:
# trainTable

In [ ]:
# table = valTable.root.HCFeats
# asv = valTable.root.labels
# acs = valTable.root.imgs

In [ ]:
# table

In [ ]:
# img

In [ ]:

# dataname="CUP-test"

# patch_size=256 #size of the tiles to extract and save in the database, must be >= to training size
# stride_size=256 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
# mirror_pad_size=128 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
# test_set_size=.3 # what percentage of the dataset should be used as a held out validation/testing set
# resize=1 #resize input images
# # class_names=["Colon","Esophagus", "Breast",  "Pancreas"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes
# class_names=["Pri","Met"]
# storage={} #holder for future pytables

# block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

# filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed

# files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\PvM\\WSI\\**\\*.png')

# random.shuffle(files)
# phases={}
# # phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))
# # phases["Test"],phases["Test2"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# phases["WSI"],phases["WSI1"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# for phase in phases.keys(): #now for each of the phases, we'll loop through the files
# #     print(phase)
    
#     totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    
#     hdf5_file = tables.open_file(f".\\{dataname}_{phase}.pytable", mode='w') #open the respective pytable
#     storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    
#     storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
#                                               shape=np.append([0],block_shape), 
#                                               chunkshape=np.append([1],block_shape),
#                                               filters=filters)
#     storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
#                                               shape=[0], 
#                                               chunkshape=[1],
#                                               filters=filters)
#     storage["HCFeats"]= hdf5_file.create_earray(hdf5_file.root, "HCFeats", img_dtype,  
#                                               shape=[0], 
#                                               chunkshape=[10],
#                                               filters=filters)

    
#     for filei in tqdm_notebook(phases[phase]): #now for each of the files
#         fname=files[filei] 
        
# #         print(fname)
#         classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
#         totals[classid]+=1

#         io=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
#         interp_method=PIL.Image.BICUBIC


#         io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
#         #io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

#         #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
#         io_arr_out=sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)

#         #resize it into a ntile x patch_size x patch_size x 3
#         io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)



#         storage["imgs"].append(io_arr_out)
#         storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
#         storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        
        
#     #lastely, we should store the number of pixels
#     npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
#     npixels[:]=totals
#     hdf5_file.close()

In [ ]:
# io_arr_out.shape

In [ ]:

# dataname="CUP-test"

# patch_size=256 #size of the tiles to extract and save in the database, must be >= to training size
# stride_size=256 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
# mirror_pad_size=128 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
# test_set_size=.3 # what percentage of the dataset should be used as a held out validation/testing set
# resize=1 #resize input images
# # class_names=["Colon","Esophagus", "Breast",  "Pancreas"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes
# class_names=["Pri","Met"]
# storage={} #holder for future pytables

# block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

# filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed

# files=glob.glob('E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\PvM\\WSI\\**\\*.png')

# random.shuffle(files)
# phases={}
# # phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))
# # phases["Test"],phases["Test2"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# phases["WSI"],phases["WSI1"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=0.00001).split(files)))
# for phase in phases.keys(): #now for each of the phases, we'll loop through the files
# #     print(phase)
    
#     totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    
#     hdf5_file = tables.open_file(f".\\{dataname}_{phase}.pytable", mode='w') #open the respective pytable
#     storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    
#     storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
#                                               shape=np.append([0],block_shape), 
#                                               chunkshape=np.append([1],block_shape),
#                                               filters=filters)
#     storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
#                                               shape=[0], 
#                                               chunkshape=[1],
#                                               filters=filters)
#     storage["HCFeats"]= hdf5_file.create_earray(hdf5_file.root, "HCFeats", img_dtype,  
#                                               shape=[0], 
#                                               chunkshape=[10],
#                                               filters=filters)

    
#     for filei in tqdm_notebook(phases[phase]): #now for each of the files
#         fname=files[filei] 
        
# #         print(fname)
#         classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
#         totals[classid]+=1

#         io=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
#         interp_method=PIL.Image.BICUBIC


#         io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
#         #io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

#         #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
#         io_arr_out=sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)

#         #resize it into a ntile x patch_size x patch_size x 3
#         io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)



#         storage["imgs"].append(io_arr_out)
#         storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
#         storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        
        
#     #lastely, we should store the number of pixels
#     npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
#     npixels[:]=totals
#     hdf5_file.close()

In [ ]:
# io_arr_out.shape